In [1]:
# Step 1: Install and Import Libraries
!pip install earthengine-api tensorflow opencv-python-headless rasterio geopandas matplotlib seaborn streamlit tqdm -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 79.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 80.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.8 MB/s eta 0:00:00


In [2]:
!rm -rf ~/.config/earthengine


In [3]:
import ee
ee.Authenticate()

In [4]:
ee.Initialize(project='my-desertification-bucket')

In [5]:
# Step 1: Import Libraries and Install tqdm
import ee
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from PIL import Image
import io
import geopandas as gpd
import pandas as pd
import os
from google.colab import files
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.cluster import KMeans
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate
from tensorflow.keras.optimizers import Adam
import streamlit as st
from tqdm import tqdm
import time


In [6]:
# Step 2: Configuration Variables
NDVI_THRESHOLD = 0.03
DESERTIFICATION_CHANGE = -0.05
SAMPLE_SCALE = 100
STAT_SCALE = 500
MAX_PIXELS = 1e11
RF_NUM_TREES = 100
RF_SAMPLE_PIXELS = 10000
EVAL_SAMPLE_PIXELS = 15000  # Increased for better evaluation

In [7]:
# Step 3: Utility Functions
def export_to_drive(image, description, folder, scale, region):
    task = ee.batch.Export.image.toDrive(
        image=image,
        description=description,
        folder=folder,
        scale=scale,
        region=region,
        maxPixels=MAX_PIXELS
    )
    task.start()
    print(f"Export task started: {description}. Check Google Drive folder '{folder}' for '{description}.tif'.")
    return task

def wait_for_task(task, timeout=600):
    start_time = time.time()
    while task.active():
        if time.time() - start_time > timeout:
            print(f"Task {task} timed out after {timeout} seconds.")
            raise SystemExit
        time.sleep(10)
    status = task.status()['state']
    print(f"Task {task} completed with status: {status}")
    if status != 'COMPLETED':
        error_message = task.status().get('error_message', 'No error message provided.')
        print(f"Task failed with status: {status}. Error: {error_message}")
        raise SystemExit
    return status

In [8]:
# Step 4: Authenticate and Initialize Google Earth Engine
def initialize_gee(project_id='my-desertification-bucket'):
    try:
        ee.Initialize(project=project_id)
        print("GEE initialized successfully.")
    except Exception as e:
        print(f"Error initializing GEE: {e}")
        print("1. Ensure you have a Google Cloud Project with Earth Engine API enabled.")
        print("2. Run the following in a separate cell to authenticate:")
        print("```python\n!rm -rf ~/.config/earthengine\nimport ee\nee.Authenticate()\n```")
        print("3. Then initialize with your project ID:")
        print("```python\nimport ee\nee.Initialize(project='my-desertification-bucket')\n```")
        print("Visit http://goo.gle/ee-auth for details.")
        raise SystemExit

initialize_gee(project_id='my-desertification-bucket')

GEE initialized successfully.


In [9]:

# Step 5: User-Defined Region of Interest
print("Enter the coordinates for your region of interest (rectangle).")
print("Default (Thar Desert, India): [68.0, 24.0, 71.0, 27.0]")
min_lon = float(input("Minimum Longitude (e.g., 68.0): ") or 68.0)
min_lat = float(input("Minimum Latitude (e.g., 24.0): ") or 24.0)
max_lon = float(input("Maximum Longitude (e.g., 71.0): ") or 71.0)
max_lat = float(input("Maximum Latitude (e.g., 27.0): ") or 27.0)

west, east = (min_lon, max_lon) if min_lon < max_lon else (max_lon, min_lon)
south, north = (min_lat, max_lat) if min_lat < max_lat else (max_lat, min_lat)

roi = ee.Geometry.Rectangle([west, south, east, north])
print(f"Region of Interest set to: [west: {west}, south: {south}, east: {east}, north: {north}]")

# Define land_mask based on the user-defined roi
water = ee.Image('JRC/GSW1_4/GlobalSurfaceWater').select('occurrence')
land_mask = water.lt(30)

scales = [30, 100, 500]
for scale in scales:
    try:
        pixel_count = ee.Image(1).reduceRegion(
            reducer=ee.Reducer.count(),
            geometry=roi,
            scale=scale,
            maxPixels=MAX_PIXELS
        ).getInfo().get('constant', 0)
        print(f"Estimated pixel count at {scale}m resolution: {pixel_count}")
    except ee.EEException as e:
        print(f"GEE Error estimating pixel count at {scale}m: {e}")
        raise SystemExit

Enter the coordinates for your region of interest (rectangle).
Default (Thar Desert, India): [68.0, 24.0, 71.0, 27.0]
Minimum Longitude (e.g., 68.0): 88
Minimum Latitude (e.g., 24.0): 20.9407
Maximum Longitude (e.g., 71.0): 90
Maximum Latitude (e.g., 27.0): 22.9407
Region of Interest set to: [west: 88.0, south: 20.9407, east: 90.0, north: 22.9407]
Estimated pixel count at 30m resolution: 55077303
Estimated pixel count at 100m resolution: 4958520
Estimated pixel count at 500m resolution: 198662


In [ ]:
# Step 6: Validate Region (Check for Land Area)
with tqdm(total=1, desc="Validating Region (Land Area Check)") as pbar:
    water = ee.Image('JRC/GSW1_4/GlobalSurfaceWater').select('occurrence')
    land_mask = water.lt(30)  # Relaxed from 10 to 20 to include more pixels
    try:
        land_area_task = export_to_drive(
            image=land_mask,
            description='land_area',
            folder='Desertification_Validation',
            scale=STAT_SCALE,
            region=roi
        )
        wait_for_task(land_area_task)
        print("Land area exported. Please manually calculate land percentage using the exported data.")
        land_percentage = float(input("Enter the land percentage (0-100) after calculating from exported data: "))
    except Exception as e:
        print(f"Error exporting land area: {e}")
        print("Falling back to direct computation.")
        try:
            land_area = land_mask.reduceRegion(
                reducer=ee.Reducer.sum(),
                geometry=roi,
                scale=STAT_SCALE,
                maxPixels=MAX_PIXELS
            ).getInfo().get('occurrence', 0)
            total_pixels = land_mask.reduceRegion(
                reducer=ee.Reducer.count(),
                geometry=roi,
                scale=STAT_SCALE,
                maxPixels=MAX_PIXELS
            ).getInfo().get('occurrence', 0)
            land_percentage = (land_area / total_pixels * 100) if total_pixels > 0 else 0
        except ee.EEException as e:
            print(f"GEE Error computing land percentage: {e}")
            raise SystemExit
    pbar.update(1)

print(f"Land Area Percentage in ROI: {land_percentage:.2f}%")
if land_percentage < 50:
    print("Warning: Less than 50% of the region is land. NDVI-based desertification analysis may be inaccurate.")
    print("Suggestion: Choose a region with more land (e.g., Thar Desert, India: [68.0, 24.0, 71.0, 27.0]).")
    user_choice = input("Do you want to proceed with the current region? (yes/no): ").strip().lower()
    if user_choice != 'yes':
        print("Exiting. Please rerun with a different region.")
        raise SystemExit
    else:
        print("Proceeding with analysis despite low land percentage. Results may be inaccurate.")

Validating Region (Land Area Check):   0%|          | 0/1 [00:00<?, ?it/s]

Export task started: land_area. Check Google Drive folder 'Desertification_Validation' for 'land_area.tif'.
Task <Task 3RMKA7P7TH37BGQGHLM5GS4R Type.EXPORT_IMAGE: land_area (State.UNSUBMITTED)> completed with status: COMPLETED
Land area exported. Please manually calculate land percentage using the exported data.
Enter the land percentage (0-100) after calculating from exported data: 90


Validating Region (Land Area Check): 100%|██████████| 1/1 [00:34<00:00, 34.62s/it]

Land Area Percentage in ROI: 90.00%


In [10]:
# Step 7: Load Multisource Data (Landsat 8, Sentinel-2, MODIS) and Features
print("Step 7: Loading multisource data and features...")

# Cloud masking functions
def mask_clouds_landsat(image):
    qa = image.select('QA_PIXEL')
    cloud_bit_mask = 1 << 3
    cloud_shadow_bit_mask = 1 << 5
    cirrus_bit_mask = 1 << 2
    radsat = image.select('QA_RADSAT')
    radsat_mask = radsat.eq(0)
    mask = (qa.bitwiseAnd(cloud_bit_mask).eq(0)
            .And(qa.bitwiseAnd(cloud_shadow_bit_mask).eq(0))
            .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
            .And(radsat_mask))
    return image.updateMask(mask).updateMask(land_mask)

def mask_clouds_sentinel(image):
    qa = image.select('QA60')
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11
    mask = (qa.bitwiseAnd(cloud_bit_mask).eq(0)
            .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0)))
    return image.updateMask(mask).updateMask(land_mask)

def mask_clouds_modis(image):
    qa = image.select('StateQA')
    cloud_mask = qa.bitwiseAnd(1 << 0).eq(0)
    shadow_mask = qa.bitwiseAnd(1 << 2).eq(0)
    mask = cloud_mask.And(shadow_mask)
    return image.updateMask(mask).updateMask(land_mask)

# Load land cover, precipitation, and temperature (static for the study period)
print("Loading additional features (land cover, precipitation, temperature)...")
with tqdm(total=1, desc="Loading Features") as pbar:
    try:
        # Land Cover: ESA WorldCover 2021
        land_cover = (ee.ImageCollection('ESA/WorldCover/v200')
                      .filterDate('2021-01-01', '2022-01-01')
                      .first()
                      .select('Map')
                      .clip(roi))

        # Precipitation: CHIRPS Daily, annual mean for the study period (2016-2023)
        precipitation_collection = (ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY')
                                   .filterDate('2016-01-01', '2023-12-31')
                                   .filterBounds(roi)
                                   .select('precipitation'))
        precipitation = precipitation_collection.mean().clip(roi)

        # Temperature: ERA5-Land, annual mean surface temperature (2m temperature)
        temperature_collection = (ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_AGGR')
                                 .filterDate('2016-01-01', '2023-12-31')
                                 .filterBounds(roi)
                                 .select('temperature_2m'))
        temperature = temperature_collection.mean().clip(roi)

        # Apply land mask to features
        land_cover = land_cover.updateMask(land_mask)
        precipitation = precipitation.updateMask(land_mask)
        temperature = temperature.updateMask(land_mask)

        # Debug: Check valid pixels in features
        for feature, name in [(land_cover, 'Land Cover'), (precipitation, 'Precipitation'), (temperature, 'Temperature')]:
            valid_pixel_count = feature.reduceRegion(
                reducer=ee.Reducer.count(),
                geometry=roi,
                scale=STAT_SCALE,
                maxPixels=MAX_PIXELS
            ).getInfo().get(feature.bandNames().get(0).getInfo(), 0)
            total_pixel_count = ee.Image(1).reduceRegion(
                reducer=ee.Reducer.count(),
                geometry=roi,
                scale=STAT_SCALE,
                maxPixels=MAX_PIXELS
            ).getInfo().get('constant', 0)
            valid_percentage = (valid_pixel_count / total_pixel_count) * 100 if total_pixel_count > 0 else 0
            print(f"{name}: Valid pixels = {valid_pixel_count}, Total pixels = {total_pixel_count}, Valid percentage = {valid_percentage:.2f}%")
    except ee.EEException as e:
        print(f"GEE Error loading features: {e}")
        raise ValueError("Step 7 failed: Unable to load land cover, precipitation, or temperature.")
    pbar.update(1)

# Load multisource NDVI data
years = ['2016', '2019', '2023']
landsat_ndvi = {}
sentinel_ndvi = {}
modis_ndvi = {}
ndvi_images = {}
available_years = []

for year in tqdm(years, desc="Loading Multisource Data"):
    start_date = f'{int(year) - 1}-10-01'
    end_date = f'{int(year) + 1}-05-31'

    # Landsat 8
    landsat_collection = (ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
                          .filterBounds(roi)
                          .filterDate(start_date, end_date)
                          .filter(ee.Filter.lt('CLOUD_COVER', 50))
                          .map(mask_clouds_landsat))
    try:
        landsat_count = landsat_collection.size().getInfo()
        print(f"Landsat collection for {year} contains {landsat_count} images.")
        if landsat_count == 0:
            print(f"No Landsat images found for {year}. Skipping Landsat data.")
            landsat_ndvi[year] = None
        else:
            landsat_img = landsat_collection.median()
            landsat_bands = landsat_img.bandNames().getInfo()
            print(f"Landsat Image Bands for {year}: {landsat_bands}")
            if not landsat_bands:
                print(f"Landsat image for {year} has no bands. Skipping Landsat data.")
                landsat_ndvi[year] = None
            else:
                landsat_scaled = landsat_img.select(['SR_B5', 'SR_B4']).multiply(0.0000275).add(-0.2).rename(['NIR', 'Red'])
                landsat_ndvi[year] = landsat_scaled.normalizedDifference(['NIR', 'Red']).rename('NDVI').clip(roi)
                # Debug: Check valid pixels
                valid_pixel_count = landsat_ndvi[year].reduceRegion(
                    reducer=ee.Reducer.count(),
                    geometry=roi,
                    scale=STAT_SCALE,
                    maxPixels=MAX_PIXELS
                ).getInfo().get('NDVI', 0)
                total_pixel_count = ee.Image(1).reduceRegion(
                    reducer=ee.Reducer.count(),
                    geometry=roi,
                    scale=STAT_SCALE,
                    maxPixels=MAX_PIXELS
                ).getInfo().get('constant', 0)
                valid_percentage = (valid_pixel_count / total_pixel_count) * 100 if total_pixel_count > 0 else 0
                print(f"Landsat NDVI {year}: Valid pixels = {valid_pixel_count}, Total pixels = {total_pixel_count}, Valid percentage = {valid_percentage:.2f}%")
    except ee.EEException as e:
        print(f"GEE Error loading Landsat data for {year}: {str(e)}")
        landsat_ndvi[year] = None

    # Sentinel-2
    sentinel_collection = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
                           .filterBounds(roi)
                           .filterDate(start_date, end_date)
                           .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 50))  # Relaxed from 30 to 50
                           .map(mask_clouds_sentinel))
    try:
        sentinel_count = sentinel_collection.size().getInfo()
        print(f"Sentinel-2 collection for {year} contains {sentinel_count} images.")
        if sentinel_count == 0:
            print(f"No Sentinel-2 images found for {year}. Skipping Sentinel-2 data.")
            sentinel_ndvi[year] = None
        else:
            sentinel_img = sentinel_collection.median()
            sentinel_bands = sentinel_img.bandNames().getInfo()
            print(f"Sentinel-2 Image Bands for {year}: {sentinel_bands}")
            if not sentinel_bands:
                print(f"Sentinel-2 image for {year} has no bands. Skipping Sentinel-2 data.")
                sentinel_ndvi[year] = None
            else:
                sentinel_scaled = sentinel_img.select(['B8', 'B4']).multiply(0.0001).rename(['NIR', 'Red'])
                sentinel_ndvi[year] = sentinel_scaled.normalizedDifference(['NIR', 'Red']).rename('NDVI').clip(roi)
                # Debug: Check valid pixels
                valid_pixel_count = sentinel_ndvi[year].reduceRegion(
                    reducer=ee.Reducer.count(),
                    geometry=roi,
                    scale=STAT_SCALE,
                    maxPixels=MAX_PIXELS
                ).getInfo().get('NDVI', 0)
                total_pixel_count = ee.Image(1).reduceRegion(
                    reducer=ee.Reducer.count(),
                    geometry=roi,
                    scale=STAT_SCALE,
                    maxPixels=MAX_PIXELS
                ).getInfo().get('constant', 0)
                valid_percentage = (valid_pixel_count / total_pixel_count) * 100 if total_pixel_count > 0 else 0
                print(f"Sentinel-2 NDVI {year}: Valid pixels = {valid_pixel_count}, Total pixels = {total_pixel_count}, Valid percentage = {valid_percentage:.2f}%")
    except ee.EEException as e:
        print(f"GEE Error loading Sentinel-2 data for {year}: {str(e)}")
        sentinel_ndvi[year] = None

    # MODIS
    modis_collection = (ee.ImageCollection("MODIS/061/MOD09A1")
                        .filterBounds(roi)
                        .filterDate(start_date, end_date)
                        .map(mask_clouds_modis))
    try:
        modis_count = modis_collection.size().getInfo()
        print(f"MODIS collection for {year} contains {modis_count} images.")
        if modis_count == 0:
            print(f"No MODIS images found for {year}. Skipping MODIS data.")
            modis_ndvi[year] = None
        else:
            modis_img = modis_collection.median()
            modis_bands = modis_img.bandNames().getInfo()
            print(f"MODIS Image Bands for {year}: {modis_bands}")
            if not modis_bands:
                print(f"MODIS image for {year} has no bands. Skipping MODIS data.")
                modis_ndvi[year] = None
            else:
                modis_scaled = modis_img.select(['sur_refl_b02', 'sur_refl_b01']).multiply(0.0001).rename(['NIR', 'Red'])
                modis_ndvi[year] = modis_scaled.normalizedDifference(['NIR', 'Red']).rename('NDVI').clip(roi)
                # Debug: Check valid pixels
                valid_pixel_count = modis_ndvi[year].reduceRegion(
                    reducer=ee.Reducer.count(),
                    geometry=roi,
                    scale=STAT_SCALE,
                    maxPixels=MAX_PIXELS
                ).getInfo().get('NDVI', 0)
                total_pixel_count = ee.Image(1).reduceRegion(
                    reducer=ee.Reducer.count(),
                    geometry=roi,
                    scale=STAT_SCALE,
                    maxPixels=MAX_PIXELS
                ).getInfo().get('constant', 0)
                valid_percentage = (valid_pixel_count / total_pixel_count) * 100 if total_pixel_count > 0 else 0
                print(f"MODIS NDVI {year}: Valid pixels = {valid_pixel_count}, Total pixels = {total_pixel_count}, Valid percentage = {valid_percentage:.2f}%")
    except ee.EEException as e:
        print(f"GEE Error loading MODIS data for {year}: {str(e)}")
        modis_ndvi[year] = None

    # Combine NDVI data: Prioritize Sentinel-2 > Landsat 8 > MODIS
    sources_available = sum(1 for ndvi in [landsat_ndvi.get(year), sentinel_ndvi.get(year), modis_ndvi.get(year)] if ndvi is not None)
    if sources_available >= 2:
        available_years.append(year)
        print(f"Year {year} added to available years with {sources_available} data sources.")
        # Create composite NDVI: Use Sentinel-2 if available, then Landsat, then MODIS
        if sentinel_ndvi.get(year) is not None:
            ndvi_images[year] = sentinel_ndvi[year]
            print(f"Using Sentinel-2 NDVI for {year}.")
        elif landsat_ndvi.get(year) is not None:
            ndvi_images[year] = landsat_ndvi[year]
            print(f"Using Landsat NDVI for {year} (Sentinel-2 unavailable).")
        else:
            ndvi_images[year] = modis_ndvi[year]
            print(f"Using MODIS NDVI for {year} (Sentinel-2 and Landsat unavailable).")
        # Debug: Check valid pixels in the composite
        valid_pixel_count = ndvi_images[year].reduceRegion(
            reducer=ee.Reducer.count(),
            geometry=roi,
            scale=STAT_SCALE,
            maxPixels=MAX_PIXELS
        ).getInfo().get('NDVI', 0)
        total_pixel_count = ee.Image(1).reduceRegion(
            reducer=ee.Reducer.count(),
            geometry=roi,
            scale=STAT_SCALE,
            maxPixels=MAX_PIXELS
        ).getInfo().get('constant', 0)
        valid_percentage = (valid_pixel_count / total_pixel_count) * 100 if total_pixel_count > 0 else 0
        print(f"Composite NDVI {year}: Valid pixels = {valid_pixel_count}, Total pixels = {total_pixel_count}, Valid percentage = {valid_percentage:.2f}%")
    else:
        print(f"Year {year} skipped: Only {sources_available} data source(s) available. Need at least 2.")

if len(available_years) < 2:
    print("Error: At least two years of NDVI data are required to compute NDVI change.")
    print("Available years with data:", available_years)
    print("Suggestions:")
    print("1. Choose a larger region (current region might be too small or over water).")
    print("2. Use a different date range (e.g., ['2015', '2020']).")
    print("3. Select a region with known coverage (e.g., Thar Desert, India: [68.0, 24.0, 71.0, 27.0]).")
    raise ValueError("Step 7 failed: Insufficient years with data.")

# Assign NDVI images for before, mid, and after
ndvi_before = ndvi_images[available_years[0]]
ndvi_mid = ndvi_images[available_years[len(available_years)//2]] if len(available_years) > 2 else None
ndvi_after = ndvi_images[available_years[-1]]

print("Step 7 completed: Multisource data and features loaded successfully.")
print(f"Available years: {available_years}")
print(f"NDVI Before ({available_years[0]}): {ndvi_before.bandNames().getInfo()}")
if ndvi_mid is not None:
    print(f"NDVI Mid ({available_years[len(available_years)//2]}): {ndvi_mid.bandNames().getInfo()}")
print(f"NDVI After ({available_years[-1]}): {ndvi_after.bandNames().getInfo()}")
print(f"Land Cover: {land_cover.bandNames().getInfo()}")
print(f"Precipitation: {precipitation.bandNames().getInfo()}")
print(f"Temperature: {temperature.bandNames().getInfo()}")

Step 7: Loading multisource data and features...
Loading additional features (land cover, precipitation, temperature)...


Loading Features:   0%|          | 0/1 [00:00<?, ?it/s]

Land Cover: Valid pixels = 46348, Total pixels = 198662, Valid percentage = 23.33%
Precipitation: Valid pixels = 46388, Total pixels = 198662, Valid percentage = 23.35%


Loading Features: 100%|██████████| 1/1 [01:23<00:00, 83.44s/it]


Temperature: Valid pixels = 44960, Total pixels = 198662, Valid percentage = 22.63%


...

Loading Multisource Data:  67%|██████▋   | 2/3 [01:02<00:34, 34.68s/it]

MODIS NDVI 2019: Valid pixels = 46398, Total pixels = 198662, Valid percentage = 23.36%
Year 2019 added to available years with 3 data sources.
Using Sentinel-2 NDVI for 2019.
Composite NDVI 2019: Valid pixels = 46398, Total pixels = 198662, Valid percentage = 23.36%
Landsat collection for 2023 contains 145 images.
Landsat Image Bands for 2023: ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'SR_QA_AEROSOL', 'ST_B10', 'ST_ATRAN', 'ST_CDIST', 'ST_DRAD', 'ST_EMIS', 'ST_EMSD', 'ST_QA', 'ST_TRAD', 'ST_URAD', 'QA_PIXEL', 'QA_RADSAT']
Landsat NDVI 2023: Valid pixels = 46398, Total pixels = 198662, Valid percentage = 23.36%
Sentinel-2 collection for 2023 contains 1287 images.
Sentinel-2 Image Bands for 2023: ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12', 'AOT', 'WVP', 'SCL', 'TCI_R', 'TCI_G', 'TCI_B', 'MSK_CLDPRB', 'MSK_SNWPRB', 'QA10', 'QA20', 'QA60', 'MSK_CLASSI_OPAQUE', 'MSK_CLASSI_CIRRUS', 'MSK_CLASSI_SNOW_ICE']
Sentinel-2 NDVI 2023: Valid pixe

Loading Multisource Data: 100%|██████████| 3/3 [01:41<00:00, 33.95s/it]

Composite NDVI 2023: Valid pixels = 46398, Total pixels = 198662, Valid percentage = 23.36%
Step 7 completed: Multisource data and features loaded successfully.
Available years: ['2016', '2019', '2023']
NDVI Before (2016): ['NDVI']


NDVI Mid (2019): ['NDVI']
NDVI After (2023): ['NDVI']
Land Cover: ['Map']
Precipitation: ['precipitation']
Temperature: ['temperature_2m']


In [11]:
# Step 8: Multisource Fusion (Weighted Average NDVI)
def prepare_ndvi_stack(landsat_ndvi, sentinel_ndvi, modis_ndvi, available_years):
    with tqdm(total=len(available_years), desc="Fusing NDVI Data") as pbar:
        fused_ndvi = {}
        for year in available_years:
            ndvi_layers = []
            weights = []
            if landsat_ndvi.get(year):
                ndvi_layers.append(landsat_ndvi[year])
                weights.append(0.4)
            if sentinel_ndvi.get(year):
                ndvi_layers.append(sentinel_ndvi[year])
                weights.append(0.4)
            if modis_ndvi.get(year):
                ndvi_layers.append(modis_ndvi[year])
                weights.append(0.2)

            total_weight = sum(weights)
            weights = [w / total_weight for w in weights] if total_weight > 0 else weights

            fused = ee.Image(0)
            for layer, weight in zip(ndvi_layers, weights):
                fused = fused.add(layer.multiply(weight))
            fused_ndvi[year] = fused.rename(f'NDVI_{year}')
            pbar.update(1)
    available_years.sort()
    ndvi_before = fused_ndvi[available_years[0]]
    ndvi_mid = fused_ndvi[available_years[len(available_years)//2]] if len(available_years) > 2 else None
    ndvi_after = fused_ndvi[available_years[-1]]
    ndvi_change = ndvi_after.subtract(ndvi_before).rename('NDVI_Change')
    return ndvi_before, ndvi_mid, ndvi_after, ndvi_change, available_years

try:
    ndvi_before, ndvi_mid, ndvi_after, ndvi_change, available_years = prepare_ndvi_stack(
        landsat_ndvi, sentinel_ndvi, modis_ndvi, available_years
    )
    print(f"NDVI change computed between {available_years[0]} and {available_years[-1]}.")
except Exception as e:
    print(f"Error in Step 8: {e}")
    print("Ensure that landsat_ndvi, sentinel_ndvi, modis_ndvi, and available_years are defined from Step 7.")
    raise SystemExit

Fusing NDVI Data: 100%|██████████| 3/3 [00:00<00:00, 1401.68it/s]

NDVI change computed between 2016 and 2023.


In [12]:
# Step 9: NDVI Change Analysis
with tqdm(total=1, desc="Computing NDVI Change") as pbar:
    # Compute NDVI change (e.g., 2023 - 2016)
    ndvi_change = ndvi_after.subtract(ndvi_before).rename('NDVI_Change')

    # Compute statistics for NDVI change
    ndvi_change_stats = ndvi_change.reduceRegion(
        reducer=ee.Reducer.mean().combine(
            reducer2=ee.Reducer.stdDev(),
            sharedInputs=True
        ),
        geometry=roi,
        scale=STAT_SCALE,
        maxPixels=MAX_PIXELS
    ).getInfo()

    mean_ndvi_change = ndvi_change_stats.get('NDVI_Change_mean', 0)
    std_dev_ndvi_change = ndvi_change_stats.get('NDVI_Change_stdDev', 0)

    print(f"Mean NDVI Change: {mean_ndvi_change}")
    print(f"Std Dev NDVI Change: {std_dev_ndvi_change}")

    # Calculate dynamic desertification threshold (current method)
    threshold = mean_ndvi_change - std_dev_ndvi_change
    print(f"Dynamic Desertification Threshold: {threshold}")

    # Classify pixels as desertified (1) or non-desertified (0) based on NDVI change
    desertification = ndvi_change.lt(threshold).rename('desertification')

    pbar.update(1)

Computing NDVI Change: 100%|██████████| 1/1 [00:34<00:00, 34.71s/it]

Mean NDVI Change: 0.020366502400826363
Std Dev NDVI Change: 0.04546757971704766
Dynamic Desertification Threshold: -0.025101077316221295


In [13]:
# Step 10: Supervised Machine Learning (Random Forest) for Desertification Classification
def train_rf_classifier(ndvi_before, ndvi_mid, ndvi_after, ndvi_change, land_cover, precipitation, temperature, roi, available_years):
    print("Starting Step 10: Training Random Forest classifier...")

    # Prepare feature stack
    print("Preparing feature stack...")
    feature_stack_bands = [ndvi_before.rename('NDVI_' + available_years[0])]
    if ndvi_mid is not None:
        feature_stack_bands.append(ndvi_mid.rename('NDVI_' + available_years[len(available_years)//2]))
    feature_stack_bands.append(ndvi_after.rename('NDVI_' + available_years[-1]))
    feature_stack_bands.extend([land_cover.rename('land_cover'), precipitation.rename('precipitation'), temperature.rename('temperature')])
    feature_stack = ee.Image.cat(feature_stack_bands).clip(roi)

    # Estimate pixel count to guide scale selection
    print("Estimating pixel count...")
    try:
        pixel_count = ee.Image(1).reduceRegion(
            reducer=ee.Reducer.count(),
            geometry=roi,
            scale=500,
            maxPixels=MAX_PIXELS
        ).getInfo().get('constant', 0)
        print(f"Estimated pixel count at 500m resolution: {pixel_count}")
        if pixel_count > 1e7:
            print("Warning: High pixel count may cause timeouts or memory limits to be exceeded. Consider reducing ROI size or increasing scale.")
    except ee.EEException as e:
        print(f"GEE Error estimating pixel count: {e}")
        print("Proceeding with caution; computation may fail due to memory limits.")

    # Check for valid pixels in ndvi_change
    print("Checking for valid pixels in ndvi_change...")
    try:
        valid_pixel_count = ndvi_change.reduceRegion(
            reducer=ee.Reducer.count(),
            geometry=roi,
            scale=500,
            maxPixels=MAX_PIXELS
        ).getInfo().get('NDVI_Change', 0)
        print(f"Number of valid pixels in ndvi_change: {valid_pixel_count}")
        if valid_pixel_count == 0:
            print("Error: No valid pixels in ndvi_change image. Cannot compute statistics.")
            print("Suggestions:")
            print("1. Check if the land mask is too strict (e.g., relax water.lt(20) further).")
            print("2. Use a smaller ROI to ensure valid data (e.g., [69.0, 25.0, 70.0, 26.0]).")
            print("3. Verify the NDVI data sources for masking issues.")
            print("Falling back to default DESERTIFICATION_CHANGE threshold.")
            dynamic_threshold = DESERTIFICATION_CHANGE
        else:
            try:
                ndvi_stats = ndvi_change.reduceRegion(
                    reducer=ee.Reducer.mean().combine(ee.Reducer.stdDev(), None, True),
                    geometry=roi,
                    scale=500,
                    maxPixels=MAX_PIXELS
                ).getInfo()
                print(f"NDVI Change Statistics Raw Output: {ndvi_stats}")
                if ndvi_stats is None or not ndvi_stats:
                    print("Error: ndvi_stats is None or empty. Falling back to default threshold.")
                    dynamic_threshold = DESERTIFICATION_CHANGE
                else:
                    mean_change = ndvi_stats.get('NDVI_Change_mean', 0)
                    std_change = ndvi_stats.get('NDVI_Change_stdDev', 0)
                    if mean_change is None or std_change is None:
                        print("Error: Mean or stdDev is None. Falling back to default threshold.")
                        dynamic_threshold = DESERTIFICATION_CHANGE
                    else:
                        dynamic_threshold = mean_change - std_change
                        print("NDVI Change Statistics:")
                        print(f"Mean NDVI Change: {mean_change}")
                        print(f"Std Dev NDVI Change: {std_change}")
                        print(f"Dynamic Desertification Threshold: {dynamic_threshold}")
            except ee.EEException as e:
                print(f"GEE Error computing NDVI change statistics: {e}")
                print("Using default DESERTIFICATION_CHANGE threshold.")
                dynamic_threshold = DESERTIFICATION_CHANGE
    except ee.EEException as e:
        print(f"GEE Error checking valid pixels in ndvi_change: {e}")
        print("Using default DESERTIFICATION_CHANGE threshold.")
        dynamic_threshold = DESERTIFICATION_CHANGE

    # Generate synthetic labels with additional complexity
    print("Generating synthetic labels...")
    labels = (ndvi_change.lt(dynamic_threshold)
              .And(precipitation.lt(300))
              .rename('Desertification'))

    # Check label distribution
    print("Checking label distribution...")
    label_histogram = None
    print("Attempting direct computation of label distribution...")
    try:
        label_stats = labels.reduceRegion(
            reducer=ee.Reducer.frequencyHistogram(),
            geometry=roi,
            scale=STAT_SCALE,
            maxPixels=MAX_PIXELS
        ).getInfo()
        label_histogram = label_stats.get('Desertification', {})
        print("Label Distribution (0 = non-desertified, 1 = desertified):", label_histogram)
    except ee.EEException as e:
        print(f"GEE Error computing label distribution directly: {e}")
        print("Falling back to export method as a last resort...")

    if label_histogram is None:
        try:
            label_task = export_to_drive(
                image=labels,
                description='label_distribution',
                folder='Desertification_Training',
                scale=STAT_SCALE,
                region=roi
            )
            wait_for_task(label_task, timeout=1800)
            print("Label distribution exported. Please compute histogram manually and enter counts.")
            count_0 = float(input("Enter the count for class 0 (non-desertified): "))
            count_1 = float(input("Enter the count for class 1 (desertified): "))
            label_histogram = {'0': count_0, '1': count_1}
        except Exception as e:
            print(f"Error exporting label distribution: {e}")
            print("Both direct computation and export failed.")
            raise SystemExit("Failed to compute label distribution.")

    if '0' not in label_histogram or '1' not in label_histogram:
        print("Error: Training data contains only one class. Cannot train classifier.")
        print("Suggestions:")
        print("1. Use a different ROI with known desertification (e.g., Thar Desert, India: [68.0, 24.0, 71.0, 27.0]).")
        print("2. Extend the time period to capture more significant changes (e.g., 2010 to 2023).")
        raise SystemExit("Training data invalid: only one class present.")

    # Sample training data
    print("Sampling training data...")
    with tqdm(total=1, desc="Sampling Training Data for Random Forest") as pbar:
        try:
            training_sample = feature_stack.addBands(labels).stratifiedSample(
                numPoints=RF_SAMPLE_PIXELS,
                classBand='Desertification',
                region=roi,
                scale=500,
                seed=42,
                classValues=[0, 1],
                classPoints=[int(RF_SAMPLE_PIXELS * label_histogram['0'] / (label_histogram['0'] + label_histogram['1'])),
                             int(RF_SAMPLE_PIXELS * label_histogram['1'] / (label_histogram['0'] + label_histogram['1']))]
            )
        except ee.EEException as e:
            print(f"GEE Error sampling training data for Random Forest: {e}")
            print("Suggestions:")
            print("1. Reduce region size (current region might be too large).")
            print("2. Increase scale to 1000 or reduce numPixels to 500.")
            raise SystemExit("Failed to sample training data.")
        pbar.update(1)

    # Convert sample to FeatureCollection for training
    print("Converting sample to FeatureCollection...")
    feature_names = ['NDVI_' + available_years[0]]
    if ndvi_mid is not None:
        feature_names.append('NDVI_' + available_years[len(available_years)//2])
    feature_names.extend(['NDVI_' + available_years[-1], 'land_cover', 'precipitation', 'temperature'])
    training_data = training_sample.select(feature_names + ['Desertification'])

    # Train Random Forest
    print("Training Random Forest classifier...")
    with tqdm(total=1, desc="Training Random Forest") as pbar:
        classifier = ee.Classifier.smileRandomForest(
            numberOfTrees=RF_NUM_TREES,
            seed=42
        ).train(
            features=training_data,
            classProperty='Desertification',
            inputProperties=feature_names
        )
        pbar.update(1)

    # Apply classifier to the entire image
    print("Applying Random Forest classifier to the entire image...")
    desertification = None
    with tqdm(total=1, desc="Applying Random Forest Classifier") as pbar:
        try:
            desertification = feature_stack.classify(classifier, 'RF_Prediction')
        except ee.EEException as e:
            print(f"GEE Error applying Random Forest classifier: {e}")
            print("Suggestions:")
            print("1. Reduce region size (current region might be too large).")
            print("2. Simplify feature stack by selecting fewer bands.")
            raise SystemExit("Failed to apply Random Forest classifier.")
        pbar.update(1)

    # Sample evaluation data and apply classifier to the sample
    print("Sampling evaluation data and applying classifier...")
    with tqdm(total=1, desc="Sampling Evaluation Data for Random Forest") as pbar:
        try:
            # Step 1: Sample features and labels
            eval_sample = feature_stack.addBands(labels).stratifiedSample(
                numPoints=1000,
                classBand='Desertification',
                region=roi,
                scale=500,
                seed=123,
                classValues=[0, 1],
                classPoints=[int(1000 * label_histogram['0'] / (label_histogram['0'] + label_histogram['1'])),
                             int(1000 * label_histogram['1'] / (label_histogram['0'] + label_histogram['1']))]
            )
            # Step 2: Apply the classifier to the sampled FeatureCollection
            eval_sample_classified = eval_sample.classify(classifier, 'RF_Prediction')
            # Step 3: Retrieve the classified sample
            eval_sample_info = eval_sample_classified.getInfo()
            # Step 4: Extract true labels and predictions
            y_true = [f['properties']['Desertification'] for f in eval_sample_info['features']]
            y_pred = [f['properties']['RF_Prediction'] for f in eval_sample_info['features']]
            print("Random Forest Classification Report (Evaluation Sample):")
            print(classification_report(y_true, y_pred))
            print("Confusion Matrix:")
            print(confusion_matrix(y_true, y_pred))
            auc = roc_auc_score(y_true, y_pred)
            print(f"ROC AUC Score: {auc:.2f}")
        except ee.EEException as e:
            print(f"GEE Error evaluating Random Forest: {e}")
            print("Skipping evaluation due to memory limits. Proceeding with classification results.")
        except KeyError as e:
            print(f"KeyError during evaluation: {e}")
            print("The evaluation sample does not contain the expected 'RF_Prediction' property.")
            print("Skipping evaluation and proceeding with classification results.")
        pbar.update(1)

    if desertification is None:
        raise SystemExit("Error: 'desertification' variable was not defined after classification.")

    print("Step 10 completed successfully.")
    return desertification

# Call Step 10 with error handling
print("Executing Step 10...")
desertification = None
try:
    desertification = train_rf_classifier(
        ndvi_before, ndvi_mid, ndvi_after, ndvi_change, land_cover, precipitation, temperature, roi, available_years
    )
    print("Step 10 completed: Random Forest classification finished. 'desertification' variable defined.")
except Exception as e:
    print(f"Error in Step 10: {e}")
    print("Ensure that Steps 8 and 9 have been executed successfully.")
    raise SystemExit("Step 10 failed to complete.")

Executing Step 10...
Starting Step 10: Training Random Forest classifier...
Preparing feature stack...
Estimating pixel count...
Estimated pixel count at 500m resolution: 198662
Checking for valid pixels in ndvi_change...
Number of valid pixels in ndvi_change: 46398
NDVI Change Statistics Raw Output: {'NDVI_Change_mean': 0.020366502400826363, 'NDVI_Change_stdDev': 0.04546757971704766}
NDVI Change Statistics:
Mean NDVI Change: 0.020366502400826363
Std Dev NDVI Change: 0.04546757971704766
Dynamic Desertification Threshold: -0.025101077316221295
Generating synthetic labels...
Checking label distribution...
Attempting direct computation of label distribution...
Label Distribution (0 = non-desertified, 1 = desertified): {'0': 40382.70588235284, '1': 5864.4745098039275}
Sampling training data...


Sampling Training Data for Random Forest: 100%|██████████| 1/1 [00:00<00:00, 1788.62it/s]


Converting sample to FeatureCollection...
Training Random Forest classifier...


Training Random Forest: 100%|██████████| 1/1 [00:00<00:00, 5737.76it/s]


Applying Random Forest classifier to the entire image...


Applying Random Forest Classifier: 100%|██████████| 1/1 [00:00<00:00, 4777.11it/s]


Sampling evaluation data and applying classifier...


Sampling Evaluation Data for Random Forest: 100%|██████████| 1/1 [01:05<00:00, 65.23s/it]

Random Forest Classification Report (Evaluation Sample):
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       873
           1       0.99      0.87      0.93       126

    accuracy                           0.98       999
   macro avg       0.99      0.94      0.96       999
weighted avg       0.98      0.98      0.98       999

Confusion Matrix:
[[872   1]
 [ 16 110]]
ROC AUC Score: 0.94
Step 10 completed successfully.
Step 10 completed: Random Forest classification finished. 'desertification' variable defined.


In [14]:
# Step 11: Export Final Desertification Map and Compute Statistics
print("Starting Step 11: Exporting final desertification map and computing statistics...")

# Verify that the desertification variable is defined
if 'desertification' not in globals():
    print("Error in Step 11: 'desertification' variable is not defined.")
    print("Ensure that Step 10 has been executed successfully.")
    raise SystemExit("Stopping execution due to missing variable.")

# Compute the percentage of desertified area
print("Computing desertification statistics...")
try:
    area_stats = desertification.reduceRegion(
        reducer=ee.Reducer.frequencyHistogram(),
        geometry=roi,
        scale=1000,  # Use a larger scale to reduce memory usage
        maxPixels=1e11
    ).getInfo()
    desertification_histogram = area_stats.get('RF_Prediction', {})
    print("Desertification Distribution (0 = non-desertified, 1 = desertified):", desertification_histogram)

    total_pixels = sum(desertification_histogram.values())
    desertified_pixels = desertification_histogram.get('1', 0)
    desertified_percentage = (desertified_pixels / total_pixels) * 100 if total_pixels > 0 else 0
    print(f"Percentage of area desertified: {desertified_percentage:.2f}%")
except ee.EEException as e:
    print(f"GEE Error computing desertification statistics: {e}")
    print("Proceeding with export, but statistics will not be available.")
    desertified_percentage = None

# Export the desertification map to Google Drive
print("Exporting desertification map to Google Drive...")
with tqdm(total=1, desc="Exporting Desertification Map") as pbar:
    export_scale = 1000  # Use a larger scale to reduce memory usage
    try:
        task = export_to_drive(
            image=desertification,
            description='final_desertification_map',
            folder='Desertification_Results',
            scale=export_scale,
            region=roi
        )
        wait_for_task(task, timeout=1800)  # 30-minute timeout
        print("Desertification map exported to Drive. Download 'final_desertification_map.tif' from the 'Desertification_Results' folder.")
    except Exception as e:
        print(f"Error exporting desertification map: {e}")
        print("Suggestions:")
        print("1. Ensure your GEE project has write access to Google Drive.")
        print("2. Reduce the region size (e.g., [69.0, 25.0, 70.0, 26.0]).")
        print("3. Increase the export scale further (e.g., scale=2000).")
        print("4. Check your GEE export quotas and permissions.")
        raise SystemExit("Failed to export desertification map.")
    pbar.update(1)

print("Step 11 completed: Desertification map exported and statistics computed.")
print("Project completed successfully!")

Starting Step 11: Exporting final desertification map and computing statistics...
Computing desertification statistics...
Desertification Distribution (0 = non-desertified, 1 = desertified): {'0': 12911.392156862776, '1': 1160.2392156862745}
Percentage of area desertified: 8.25%
Exporting desertification map to Google Drive...


Exporting Desertification Map:   0%|          | 0/1 [00:00<?, ?it/s]

Export task started: final_desertification_map. Check Google Drive folder 'Desertification_Results' for 'final_desertification_map.tif'.


Exporting Desertification Map: 100%|██████████| 1/1 [02:53<00:00, 173.25s/it]

Task <Task 3YCVNUFFOBUXMWR2EKKPZ7EA Type.EXPORT_IMAGE: final_desertification_map (State.UNSUBMITTED)> completed with status: COMPLETED
Desertification map exported to Drive. Download 'final_desertification_map.tif' from the 'Desertification_Results' folder.
Step 11 completed: Desertification map exported and statistics computed.
Project completed successfully!


In [15]:
# Required imports for displaying images
from IPython.display import Image, display

# Step 12: Print Detailed Desertification Results and Display Thumbnail Images in the Notebook
print("Starting Step 12: Printing detailed desertification results and displaying thumbnail images for the user-specified region...")

# Print the current ROI coordinates for clarity
roi_bounds = roi.bounds().coordinates().getInfo()[0]
print(f"Region of Interest (ROI) Coordinates: [{roi_bounds[0][0]:.1f}, {roi_bounds[0][1]:.1f}, {roi_bounds[2][0]:.1f}, {roi_bounds[2][1]:.1f}]")

# Verify that required variables are defined
required_vars = ['ndvi_before', 'ndvi_mid', 'ndvi_after', 'desertification', 'roi', 'available_years']
missing_vars = [var for var in required_vars if var not in globals()]
if missing_vars:
    print(f"Error in Step 12: The following variables are not defined: {missing_vars}")
    print("Ensure that Steps 8, 9, and 10 have been executed successfully.")
    raise SystemExit("Stopping execution due to missing variables.")

# 1. Summarize All Images (Text-Based)
print("\n1. Summary of All Images (Text-Based)")
print("-------------------------------------")

# Function to compute image statistics
def compute_image_stats(image, band_name, geometry, scale, max_pixels):
    try:
        valid_pixels = image.reduceRegion(
            reducer=ee.Reducer.count(),
            geometry=geometry,
            scale=scale,
            maxPixels=max_pixels
        ).getInfo().get(band_name, 0)

        total_pixels = ee.Image(1).reduceRegion(
            reducer=ee.Reducer.count(),
            geometry=geometry,
            scale=scale,
            maxPixels=max_pixels
        ).getInfo().get('constant', 1)

        percent_valid = (valid_pixels / total_pixels * 100) if total_pixels > 0 else 0

        mean_value = image.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=geometry,
            scale=scale,
            maxPixels=max_pixels
        ).getInfo().get(band_name, None)

        return valid_pixels, percent_valid, mean_value
    except ee.EEException as e:
        print(f"GEE Error computing stats for {band_name}: {e}")
        return None, None, None

# NDVI Images
ndvi_images = [
    (ndvi_before, f"NDVI_{available_years[0]}", "NDVI Before (2016)"),
    (ndvi_mid, f"NDVI_{available_years[len(available_years)//2]}", "NDVI Mid (2019)" if ndvi_mid is not None else "NDVI Mid (Not Available)"),
    (ndvi_after, f"NDVI_{available_years[-1]}", "NDVI After (2023)")
]

for image, band_name, label in ndvi_images:
    if image is None:
        print(f"{label}: Not available")
        continue
    valid_pixels, percent_valid, mean_value = compute_image_stats(image, band_name, roi, 1000, 1e8)
    if valid_pixels is not None:
        print(f"{label}:")
        print(f"  Valid Pixels: {valid_pixels}")
        print(f"  Percentage of Valid Pixels: {percent_valid:.2f}%")
        print(f"  Mean {band_name}: {mean_value:.3f}" if mean_value is not None else "  Mean Value: Not available")

# Desertification Image
valid_pixels, percent_valid, mean_value = compute_image_stats(desertification, 'RF_Prediction', roi, 1000, 1e8)
if valid_pixels is not None:
    print("Desertification Map (final_desertification_map.tif contents):")
    print(f"  Valid Pixels: {valid_pixels}")
    print(f"  Percentage of Valid Pixels: {percent_valid:.2f}%")
    print(f"  Mean RF_Prediction (proportion of desertified pixels): {mean_value:.3f}" if mean_value is not None else "  Mean Value: Not available")

# 2. Analyze How the Map Has Changed (Text-Based)
print("\n2. Changes in the Map (NDVI Trends and Desertification Correlation)")
print("----------------------------------------------------------------")

try:
    mean_ndvi_before = ndvi_before.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=roi,
        scale=1000,
        maxPixels=1e8
    ).getInfo().get(f"NDVI_{available_years[0]}", None)

    mean_ndvi_after = ndvi_after.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=roi,
        scale=1000,
        maxPixels=1e8
    ).getInfo().get(f"NDVI_{available_years[-1]}", None)

    if mean_ndvi_before is not None and mean_ndvi_after is not None:
        ndvi_change = mean_ndvi_after - mean_ndvi_before
        print(f"NDVI Change (2016 to 2023): {ndvi_change:.3f}")
        print("Interpretation: Negative values indicate vegetation loss, which may correlate with desertification.")
    else:
        print("NDVI Change: Not available (mean values could not be computed).")

    significant_decline = ndvi_before.subtract(ndvi_after).gt(0.1)
    decline_and_desertified = significant_decline.multiply(desertification.eq(1))
    decline_and_desertified_count = decline_and_desertified.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=roi,
        scale=1000,
        maxPixels=1e8
    ).getInfo().get('RF_Prediction', 0)

    decline_count = significant_decline.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=roi,
        scale=1000,
        maxPixels=1e8
    ).getInfo().get(f"NDVI_{available_years[0]}", 0)

    if decline_count > 0:
        percent_desertified_in_declined = (decline_and_desertified_count / decline_count) * 100
        print(f"Percentage of areas with significant NDVI decline (>0.1) that are desertified: {percent_desertified_in_declined:.2f}%")
    else:
        print("Correlation Analysis: Not available (no significant NDVI decline detected).")
except ee.EEException as e:
    print(f"GEE Error analyzing map changes: {e}")
    print("Skipping change analysis.")

# 3. Detailed Summary of the TIF File Contents (Text-Based)
print("\n3. Detailed Contents of final_desertification_map.tif (Text-Based)")
print("----------------------------------------------------------------")

try:
    desertification_stats = desertification.reduceRegion(
        reducer=ee.Reducer.frequencyHistogram(),
        geometry=roi,
        scale=1000,
        maxPixels=1e8
    ).getInfo()
    desertification_histogram = desertification_stats.get('RF_Prediction', {})
    print("Pixel Counts:")
    print(f"  Non-Desertified (0): {desertification_histogram.get('0', 0)} pixels")
    print(f"  Desertified (1): {desertification_histogram.get('1', 0)} pixels")

    pixel_area = ee.Image.pixelArea()
    area_image = pixel_area.multiply(desertification.eq(1))
    area_stats = area_image.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=roi,
        scale=1000,
        maxPixels=1e8
    ).getInfo()
    desertified_area_m2 = area_stats.get('RF_Prediction', 0)
    total_area_m2 = pixel_area.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=roi,
        scale=1000,
        maxPixels=1e8
    ).getInfo().get('area', 0)
    non_desertified_area_m2 = total_area_m2 - desertified_area_m2

    desertified_area_km2 = desertified_area_m2 / 1e6
    non_desertified_area_km2 = non_desertified_area_m2 / 1e6
    total_area_km2 = total_area_m2 / 1e6

    print("\nArea Statistics:")
    print(f"  Total Area: {total_area_km2:.2f} km²")
    print(f"  Desertified Area (Class 1): {desertified_area_km2:.2f} km²")
    print(f"  Non-Desertified Area (Class 0): {non_desertified_area_km2:.2f} km²")

    print("\nSampled 10x10 Grid of Desertification Map:")
    print("0 = Non-Desertified, 1 = Desertified")
    bounds = roi.bounds().getInfo()['coordinates'][0]
    west, south = bounds[0][0], bounds[0][1]
    east, north = bounds[2][0], bounds[2][1]
    lon_step = (east - west) / 10
    lat_step = (north - south) / 10
    points = []
    for i in range(10):
        for j in range(10):
            lon = west + (i + 0.5) * lon_step
            lat = south + (j + 0.5) * lat_step
            points.append(ee.Geometry.Point(lon, lat))
    point_collection = ee.FeatureCollection(points)

    sampled_values = desertification.sampleRegions(
        collection=point_collection,
        scale=1000,
        geometries=False
    ).getInfo()

    grid = [[0 for _ in range(10)] for _ in range(10)]
    idx = 0
    for i in range(10):
        for j in range(10):
            if idx < len(sampled_values['features']):
                grid[i][j] = sampled_values['features'][idx]['properties']['RF_Prediction']
            idx += 1

    for row in grid:
        print(" ".join(map(str, row)))
except ee.EEException as e:
    print(f"GEE Error computing desertification statistics: {e}")
    print("Skipping pixel counts, area statistics, and grid.")

# 4. Display Thumbnail Images
print("\n4. Thumbnail Images of NDVI and Desertification Maps")
print("--------------------------------------------------")
print("Note: These are low-resolution thumbnails for visualization. For detailed analysis, download 'final_desertification_map.tif' from Google Drive.")

# Create a desertification severity image based on NDVI decline
try:
    # Compute NDVI decline (positive values indicate vegetation loss)
    ndvi_decline = ndvi_before.subtract(ndvi_after).rename('decline')
    # Normalize NDVI decline to a 0-1 range for desertified areas
    max_decline = 0.3  # Maximum expected NDVI decline
    normalized_decline = ndvi_decline.divide(max_decline).clamp(0, 1)  # Scale to 0-1
    # For non-desertified areas, set value to 0; for desertified areas, scale from 0.5 to 1
    desertification_severity = desertification.multiply(normalized_decline.multiply(0.5).add(0.5)).where(desertification.eq(0), 0)
    desertification_severity = desertification_severity.rename('severity')
except ee.EEException as e:
    print(f"GEE Error computing desertification severity: {e}")
    print("Falling back to binary desertification map for visualization.")
    desertification_severity = desertification

# Function to generate and display a thumbnail
def display_thumbnail(image, band_name, label, vis_params, region):
    try:
        thumbnail_url = image.getThumbURL({
            'bands': band_name,
            'region': region,
            'min': vis_params['min'],
            'max': vis_params['max'],
            'palette': vis_params['palette'],
            'dimensions': 300,  # Small thumbnail size for notebook display
            'format': 'png'
        })
        print(f"\n{label}:")
        display(Image(url=thumbnail_url))
    except ee.EEException as e:
        print(f"GEE Error generating thumbnail for {label}: {e}")

# Visualization parameters
ndvi_vis_params = {
    'min': 0,
    'max': 1,
    'palette': ['#d73027', '#f46d43', '#fdae61', '#fee08b', '#d9ef8b', '#a6d96a', '#66bd63', '#1a9850']  # Red to Green
}

desertification_vis_params = {
    'min': 0,
    'max': 1,
    'palette': ['#228B22', '#228B22', '#FFFF00', '#FFA500', '#FF0000']  # Forest Green for non-desertified, Yellow-Orange-Red for desertified
}

# Display thumbnails for NDVI images
for image, band_name, label in ndvi_images:
    if image is not None:
        display_thumbnail(image, band_name, label, ndvi_vis_params, roi)

# Display thumbnail for desertification severity map
display_thumbnail(desertification_severity, 'severity', "Desertification Severity Map (Yellow = Low, Orange = Medium, Red = High Severity)", desertification_vis_params, roi)

print("\nStep 12 completed: Detailed results and thumbnail images displayed in the notebook for the user-specified region.")

Starting Step 12: Printing detailed desertification results and displaying thumbnail images for the user-specified region...
Region of Interest (ROI) Coordinates: [88.0, 20.9, 90.0, 22.9]

1. Summary of All Images (Text-Based)
-------------------------------------
NDVI Before (2016):
  Valid Pixels: 14533
  Percentage of Valid Pixels: 29.22%
  Mean NDVI_2016: 0.508
NDVI Mid (2019):
  Valid Pixels: 14533
  Percentage of Valid Pixels: 29.22%
  Mean NDVI_2019: 0.508
NDVI After (2023):
  Valid Pixels: 14533
  Percentage of Valid Pixels: 29.22%
  Mean NDVI_2023: 0.527
Desertification Map (final_desertification_map.tif contents):
  Valid Pixels: 14142
  Percentage of Valid Pixels: 28.44%
  Mean RF_Prediction (proportion of desertified pixels): 0.082

2. Changes in the Map (NDVI Trends and Desertification Correlation)
----------------------------------------------------------------
NDVI Change (2016 to 2023): 0.019
Interpretation: Negative values indicate vegetation loss, which may correlate 


NDVI Mid (2019):



NDVI After (2023):



Desertification Severity Map (Yellow = Low, Orange = Medium, Red = High Severity):



Step 12 completed: Detailed results and thumbnail images displayed in the notebook for the user-specified region.
